<a href="https://colab.research.google.com/github/binit-13/Machine-Learning-Notes/blob/main/Deep_learning_Hyper_parameter_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import requests
from io import StringIO

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
url = 'https://raw.githubusercontent.com/npradaschnor/Pima-Indians-Diabetes-Dataset/master/diabetes.csv'

In [ ]:
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:66.0) Gecko/20100101 Firefox/66.0"}
content = requests.get(url, headers)
data = StringIO(content.text)

In [ ]:
df = pd.read_csv(data,error_bad_lines=False)

<ipython-input-16-ec5b94c15198>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(data,error_bad_lines=False)


In [ ]:
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [ ]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [ ]:
X=df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [ ]:
scaler = StandardScaler()

In [ ]:
X_trans = scaler.fit_transform(X)

In [ ]:
X.shape

(768, 8)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=1)

In [ ]:
model = Sequential()
model.add(Dense(32, activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])


In [ ]:
model.fit(X_train,y_train,epochs=100, batch_size=32, validation_data=(X_test,y_test))

Epoch 1/100
20/20 [==============================] - 0s 7ms/step - loss: 0.7673 - accuracy: 0.6466 - val_loss: 0.7770 - val_accuracy: 0.6104
Epoch 2/100
20/20 [==============================] - 0s 8ms/step - loss: 0.7380 - accuracy: 0.6661 - val_loss: 0.7467 - val_accuracy: 0.6104
Epoch 3/100
20/20 [==============================] - 0s 9ms/step - loss: 0.7058 - accuracy: 0.6808 - val_loss: 0.7234 - val_accuracy: 0.6169
Epoch 4/100
20/20 [==============================] - 0s 7ms/step - loss: 0.7262 - accuracy: 0.6450 - val_loss: 0.7748 - val_accuracy: 0.5779
Epoch 5/100
20/20 [==============================] - 0s 4ms/step - loss: 0.7190 - accuracy: 0.6401 - val_loss: 0.7405 - val_accuracy: 0.6688
Epoch 6/100
20/20 [==============================] - 0s 5ms/step - loss: 0.6871 - accuracy: 0.6612 - val_loss: 0.6808 - val_accuracy: 0.6429
Epoch 7/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6635 - accuracy: 0.6629 - val_loss: 0.6806 - val_accuracy: 0.6558
Epoch 8/100
2

In [ ]:
# 1. selecting appropraite optimizer
# 2. no.of nodes in layer
# 3. no.of layers
# 4. All in all one model finally

In [ ]:
pip install -U keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.3/167.3 KB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 30.3 MB/s eta 0:00:00


In [ ]:
import kerastuner as kt

In [ ]:
def build_model(hp):

  model = Sequential()

  model.add(Dense(32, activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  optimizer = hp.Choice('optimizer',values = ['adam','sgd','rmsprop','adadelta'])
  model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [ ]:
tuner = kt.RandomSearch(build_model,
                        objective = 'val_accuracy',
                        max_trials = 5)


In [ ]:
 tuner.search(X_train, y_train, epochs=10,validation_data=(X_test,y_test))

In [ ]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [ ]:
model = tuner.get_best_models(num_models=1)[0] #---> best optimizer le aba model pheri train garne, this is way to get best optimizer model

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321
Trainable params: 321
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_train, y_train,batch_size=32, epochs=100, validation_data=(X_test,y_test))

Epoch 1/100
20/20 [==============================] - 3s 22ms/step - loss: 0.8625 - accuracy: 0.6140 - val_loss: 0.7972 - val_accuracy: 0.6364
Epoch 2/100
20/20 [==============================] - 0s 4ms/step - loss: 0.8092 - accuracy: 0.6384 - val_loss: 0.9196 - val_accuracy: 0.5779
Epoch 3/100
20/20 [==============================] - 0s 4ms/step - loss: 0.7597 - accuracy: 0.6482 - val_loss: 0.7162 - val_accuracy: 0.6818
Epoch 4/100
20/20 [==============================] - 0s 4ms/step - loss: 0.7279 - accuracy: 0.6515 - val_loss: 0.7028 - val_accuracy: 0.6623
Epoch 5/100
20/20 [==============================] - 0s 4ms/step - loss: 0.7106 - accuracy: 0.6401 - val_loss: 0.8019 - val_accuracy: 0.6623
Epoch 6/100
20/20 [==============================] - 0s 5ms/step - loss: 0.7235 - accuracy: 0.6612 - val_loss: 0.8341 - val_accuracy: 0.6688
Epoch 7/100
20/20 [==============================] - 0s 4ms/step - loss: 0.7066 - accuracy: 0.6678 - val_loss: 0.7825 - val_accuracy: 0.5844
Epoch 8/100


In [ ]:
# right number of neuron

def build_model(hp):
  model = Sequential()

  units = hp.Int('units', 8,128, step=8) #8 DHEKI 128 SAMMA KHOJCHA

  model.add(Dense(units=units, activation='relu', input_dim=8))
  model.add(Dense(1, activation='sigmoid'))

  model.compile(optimizer='adam',loss = 'binary_crossentropy',metrics=['accuracy'])

  return model

In [ ]:
tuner = kt.RandomSearch(build_model,objective = 'val_accuracy',max_trials=5,
                        directory='my_dir') #my dir file create huncha for future purpose without this
                        # ]tuner search nai hudaina so

In [ ]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.6558441519737244

Best val_accuracy So Far: 0.6688311696052551
Total elapsed time: 00h 00m 12s


In [ ]:
tuner.get_best_hyperparameters()[0].values

{'units': 32}

In [ ]:
model = tuner.get_best_models(num_models=1)[0]

In [ ]:
model.fit(X_train, y_train, epochs=100, batch_size=32, initial_epoch=6)

Epoch 7/100
20/20 [==============================] - 2s 3ms/step - loss: 1.6836 - accuracy: 0.6759
Epoch 8/100
20/20 [==============================] - 0s 2ms/step - loss: 1.4784 - accuracy: 0.6775
Epoch 9/100
20/20 [==============================] - 0s 2ms/step - loss: 1.3036 - accuracy: 0.6710
Epoch 10/100
20/20 [==============================] - 0s 2ms/step - loss: 1.1320 - accuracy: 0.6645
Epoch 11/100
20/20 [==============================] - 0s 2ms/step - loss: 1.0103 - accuracy: 0.6792
Epoch 12/100
20/20 [==============================] - 0s 2ms/step - loss: 0.9587 - accuracy: 0.6384
Epoch 13/100
20/20 [==============================] - 0s 2ms/step - loss: 0.8508 - accuracy: 0.6743
Epoch 14/100
20/20 [==============================] - 0s 2ms/step - loss: 0.8499 - accuracy: 0.6759
Epoch 15/100
20/20 [==============================] - 0s 2ms/step - loss: 0.8232 - accuracy: 0.6515
Epoch 16/100
20/20 [==============================] - 0s 2ms/step - loss: 0.7971 - accuracy: 0.6775
Epo

In [ ]:
# no.of layers

def build_model(hp):

  model = Sequential()

  model.add(Dense(72, activation='relu', input_dim=8))

  for i in range(hp.Int('num_layers'), min_value=1, max_value=10):
     model.add(Dense(72,activation='relu'))

  model.add(Dense(1, activation='sigmoid'))

  model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [ ]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=3,directory='mydir1')

In [ ]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test,y_test))

In [ ]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 9}

In [ ]:
model = tuner.get_best_models(num_models=1)[0]

In [ ]:
model.fit(X_train, y_train, epochs=100, batch_size=32, initial_epoch=6)

Epoch 7/100
20/20 [==============================] - 4s 4ms/step - loss: 0.5794 - accuracy: 0.7085
Epoch 8/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5780 - accuracy: 0.7003
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5829 - accuracy: 0.6922
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5663 - accuracy: 0.7199
Epoch 11/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5923 - accuracy: 0.6808
Epoch 12/100
20/20 [==============================] - 0s 5ms/step - loss: 0.5775 - accuracy: 0.6792
Epoch 13/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5691 - accuracy: 0.6938
Epoch 14/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5570 - accuracy: 0.7199
Epoch 15/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5561 - accuracy: 0.7134
Epoch 16/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5411 - accuracy: 0.7264
Epo

In [ ]:
# all at once

def build_model(hp):

  model = Sequential()

  counter = 0

  for i in range(hp.Int('num_layers',min_value=1, max_value=10)):
    if counter == 0:
      model.add(
        Dense(hp.Int('units'+str(i),min_value=8,max_value=128),
            activation = hp.Choice('activation'+str(i),values=['relu','tanh','sigmoid','elu']),
            input_dim = 8
            ))
    else:
         Dense(
            hp.Int('units'+str(i),min_value=8,max_value=128),
            activation = hp.Choice('activation'+str(i),values=['relu','tanh','sigmoid','elu'])
            )

    counter+=1

  model.add(Dense(1, activation='sigmoid'))

  model.compile(optimizer=hp.Choice('optimizer',values = ['sgd','adam','adadelta','rmsprop','nadam']),loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [ ]:
tuner = kt.RandomSearch(build_model, max_trials=3, objective='val_accuracy',directory='mydir',project_name='final')

In [ ]:
tuner.search(X_train, y_train, epochs = 5, validation_data = (X_test, y_test))

Trial 3 Complete [00h 00m 02s]
val_accuracy: 0.6168830990791321

Best val_accuracy So Far: 0.6558441519737244
Total elapsed time: 00h 00m 08s


In [ ]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 6,
 'units0': 60,
 'activation0': 'sigmoid',
 'optimizer': 'nadam',
 'units1': 45,
 'activation1': 'sigmoid',
 'units2': 65,
 'activation2': 'sigmoid',
 'units3': 89,
 'activation3': 'sigmoid',
 'units4': 104,
 'activation4': 'tanh',
 'units5': 56,
 'activation5': 'sigmoid'}